<a href="https://colab.research.google.com/github/lorenzrossi/AMD-ComicsFaces/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import os
from os import listdir
import pandas as pd
# Basic packages
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pickle

# Image preprocessing
import pathlib
import PIL
import PIL.Image
import cv2
import zipfile

from google.colab import drive
drive.mount('/content/drive')

from google.colab import files

uploaded = files.upload()

# UPLOAD YOUR KAGGLE API KEY

for fn in uploaded.keys():
  print('User uploaded key "{name}"'.format(
      name=fn, length=len(uploaded[fn])))
  
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Saving kaggle.json to kaggle.json
User uploaded key "kaggle.json"


In [18]:
import kaggle
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [19]:
# usare il link di kaggle per scaricare le immagini

!kaggle datasets download -d defileroff/comic-faces-paired-synthetic-v2

comic-faces-paired-synthetic-v2.zip: Skipping, found more recently modified local copy (use --force to force download)


In [20]:
with zipfile.ZipFile('comic-faces-paired-synthetic-v2.zip','r') as zipref:
     zipref.extractall()
!rm -rf /content/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela/samples

In [22]:
dir = "/content/face2comics_v2.0.0_by_Sxela/face2comics_v2.0.0_by_Sxela"


# I define some parameters in order resize image in an uniform way and I set 3 as number of channels of the image, since I want RGB images
img_width = 100
img_height = 100
img_size = (img_width, img_height)

# Pets is the list in which alle the images will be stored
pics = []

# I define the two categories of pics ( related to the sub-directories of comics and faces)
categories = ['comics','faces']

# I create a funciotn in order to load the data, transform the into array, assign them to a class and store them in the pics list
def create_data():

# the function iterates throught the two sub-directories
    for category in categories:
        path = os.path.join(dir, category)

# assign 0 to comics and 1 to faces, according to the index of the categories
        pic_class = categories.index(category)

# the function then iterates through each image in both the folders. I used the opencv (cv2 when imported) package to read and load them. IMREAD_COLOR keeps the color in the image, whereas IMREAD_GRAYSCALE would turn them into grayscale images.
# The function resize the images as defined by the previous parameters and finally appends the array of the image, together with the class value (creating a sub-list of two values for each image), to the pets list.
# The function skips those images which seem broken.
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path,img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, img_size)
                pics.append([new_array, pic_class])
            except Exception as e:
                pass

create_data()

print(len(pics))

# since the previous function iterates linearly through the two sub-directories, I shuffle the data in order to mix shuffle the order of the images
random.shuffle(pics)

for sample in pics[:9]:
    print(sample[1])

# split the data arrays and the labels for the classification task

X = []
y = []

for image, label in pics:
    X.append(image)
    y.append(label)

X = np.array(X).reshape(-1, img_width, img_height, 1)

# Saving the pickles in the apposite folder

pickle_out = open('/content/drive/MyDrive/ComicsFaces/Pickles/X.pickle','wb')
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open('/content/drive/MyDrive/ComicsFaces/Pickles/y.pickle','wb')
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open('/content/drive/MyDrive/ComicsFaces/Pickles/X.pickle','rb')
X = pickle.load(pickle_in)

pickle_in = open('/content/drive/MyDrive/ComicsFaces/Pickles/y.pickle','rb')
y = pickle.load(pickle_in)

20000
1
0
1
1
1
1
1
0
0
